In [1]:
#Import relevant packages
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier 
import matplotlib.pyplot as plt
from sklearn import tree

In [2]:
#Import the datasets
sub_df = pd.read_csv('submission_format.csv')
test_vals = pd.read_csv('test_set_values.csv')
train_labels = pd.read_csv('training_set_labels.csv')
train_vals = pd.read_csv('training_set_values.csv')

In [3]:
sub_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            14850 non-null  int64 
 1   status_group  14850 non-null  object
dtypes: int64(1), object(1)
memory usage: 232.2+ KB


In [4]:
test_vals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     14850 non-null  int64  
 1   amount_tsh             14850 non-null  float64
 2   date_recorded          14850 non-null  object 
 3   funder                 13981 non-null  object 
 4   gps_height             14850 non-null  int64  
 5   installer              13973 non-null  object 
 6   longitude              14850 non-null  float64
 7   latitude               14850 non-null  float64
 8   wpt_name               14850 non-null  object 
 9   num_private            14850 non-null  int64  
 10  basin                  14850 non-null  object 
 11  subvillage             14751 non-null  object 
 12  region                 14850 non-null  object 
 13  region_code            14850 non-null  int64  
 14  district_code          14850 non-null  int64  
 15  lg

In [5]:
train_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            59400 non-null  int64 
 1   status_group  59400 non-null  object
dtypes: int64(1), object(1)
memory usage: 928.2+ KB


In [6]:
train_labels['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

## Merge Training Dataset

We've elected to merge the Training datasets with each well's information and functionality into one.

In [7]:
#Merge the training datatsets
train_df = train_vals.merge(train_labels, left_on = 'id', right_on = 'id')

In [8]:
train_df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

## Data Exploration

### Check for Duplicate IDs

In [10]:
#Check if any duplicate IDs
train_df['id'].is_unique

True

No duplicate IDs in our data!

### Initial Look

In [11]:
train_df['lga'].value_counts()

Njombe          2503
Arusha Rural    1252
Moshi Rural     1251
Bariadi         1177
Rungwe          1106
                ... 
Moshi Urban       79
Kigoma Urban      71
Arusha Urban      63
Lindi Urban       21
Nyamagana          1
Name: lga, Length: 125, dtype: int64

In [12]:
train_df['ward'].value_counts()

Igosi            307
Imalinyi         252
Siha Kati        232
Mdandu           231
Nduruma          217
                ... 
Linda              1
Uchindile          1
Ukata              1
Machinjioni        1
Kihangimahuka      1
Name: ward, Length: 2092, dtype: int64

In [13]:
train_df['waterpoint_type_group'].value_counts()

communal standpipe    34625
hand pump             17488
other                  6380
improved spring         784
cattle trough           116
dam                       7
Name: waterpoint_type_group, dtype: int64

In [14]:
train_df.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [15]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

### Check the duplicate columns

In [16]:
#Who operates the waterpoint
train_df[['scheme_management', 'scheme_name']].value_counts()

scheme_management  scheme_name               
VWC                K                             571
WUA                Chalinze wate                 404
VWC                DANIDA                        378
                   M                             331
WUG                None                          325
                                                ... 
VWC                NKENJA                          1
                   NYA/ MAK/ BUK piped scheme      1
                   NYA/MAK/BUK  piped scheme       1
                   NYA/MAK/BUK piped scheme        1
Company            BFFS                            1
Length: 3077, dtype: int64

In [17]:
#The kind of extraction the waterpoint uses
train_df[['extraction_type', 'extraction_type_group', 'extraction_type_class']].value_counts()

extraction_type            extraction_type_group  extraction_type_class
gravity                    gravity                gravity                  26780
nira/tanira                nira/tanira            handpump                  8154
other                      other                  other                     6430
submersible                submersible            submersible               4764
swn 80                     swn 80                 handpump                  3670
mono                       mono                   motorpump                 2865
india mark ii              india mark ii          handpump                  2400
afridev                    afridev                handpump                  1770
ksb                        submersible            submersible               1415
other - rope pump          rope pump              rope pump                  451
other - swn 81             other handpump         handpump                   229
windmill                   wind-power

In [18]:
#How the waterpoint is managed
train_df[['management', 'management_group']].value_counts()

management        management_group
vwc               user-group          40507
wug               user-group           6515
water board       user-group           2933
wua               user-group           2535
private operator  commercial           1971
parastatal        parastatal           1768
water authority   commercial            904
other             other                 844
company           commercial            685
unknown           unknown               561
other - school    other                  99
trust             commercial             78
dtype: int64

In [19]:
#What the water costs
train_df[['payment', 'payment_type']].value_counts()

payment                payment_type
never pay              never pay       25348
pay per bucket         per bucket       8985
pay monthly            monthly          8300
unknown                unknown          8157
pay when scheme fails  on failure       3914
pay annually           annually         3642
other                  other            1054
dtype: int64

In [20]:
#The quality of the water
train_df[['water_quality', 'quality_group']].value_counts()

water_quality       quality_group
soft                good             50818
salty               salty             4856
unknown             unknown           1876
milky               milky              804
coloured            colored            490
salty abandoned     salty              339
fluoride            fluoride           200
fluoride abandoned  fluoride            17
dtype: int64

In [21]:
#The quantity of the water
train_df[['quantity', 'quantity_group']].value_counts()

quantity      quantity_group
enough        enough            33186
insufficient  insufficient      15129
dry           dry                6246
seasonal      seasonal           4050
unknown       unknown             789
dtype: int64

In [22]:
#The source of the water
train_df[['source', 'source_type', 'source_class']].value_counts()

source                source_type           source_class
spring                spring                groundwater     17021
shallow well          shallow well          groundwater     16824
machine dbh           borehole              groundwater     11075
river                 river/lake            surface          9612
rainwater harvesting  rainwater harvesting  surface          2295
hand dtw              borehole              groundwater       874
lake                  river/lake            surface           765
dam                   dam                   surface           656
other                 other                 unknown           212
unknown               other                 unknown            66
dtype: int64

In [23]:
#The kind of waterpoint
train_df[['waterpoint_type', 'waterpoint_type_group']].value_counts()

waterpoint_type              waterpoint_type_group
communal standpipe           communal standpipe       28522
hand pump                    hand pump                17488
other                        other                     6380
communal standpipe multiple  communal standpipe        6103
improved spring              improved spring            784
cattle trough                cattle trough              116
dam                          dam                          7
dtype: int64

In [24]:
#Geographic location
train_df[['subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward']].value_counts()

subvillage            region         region_code  district_code  lga           ward   
Mtaa Wa Kitunda Kati  Dar es Salaam  7            2              Ilala         Kitunda    84
Marurani Kati         Arusha         2            2              Arusha Rural  Nduruma    59
Marurani Juu          Arusha         2            2              Arusha Rural  Nduruma    54
Mtaa Wa Vikongoro     Dar es Salaam  7            2              Ilala         Chanika    47
Vikuge                Pwani          6            2              Kibaha        Soga       41
                                                                                          ..
Minazini              Lindi          80           43             Liwale        Mihumo      1
                      Morogoro       5            4              Ulanga        Itete       1
                                                  6              Mvomero       Hembeti     1
                                                                            

### Dropping Columns/Rows

In [25]:
#Drop scheme_name column as well as columns with duplicate data
clean_train = train_df.drop(['scheme_name', 'num_private', 'recorded_by', 'extraction_type', 'extraction_type_group', 
                           'management', 'payment', 'water_quality', 'quantity_group', 'source', 'waterpoint_type'], 
                          axis=1)

#Drop null values
clean_train = clean_train.dropna()


#Check the wells with Longitude = 0
clean_train.loc[clean_train['longitude'] == 0]

#Select and Keep the rows that don't have Longitude of 0
clean_train = clean_train.loc[clean_train['longitude'] != 0]

We've elected to drop the column 'scheme_name' due to the extensive number of missing values - 28,166 to be precise. 

The columns with more unique values were dropped and columns with more generalized values were kept. Rows with missing data have been dropped as well as it is a smaller portion of the dataset. 

1326 wells have a Longitude of 0. That's not right, Tanzania's Longitude ranges from around 30-40 degrees East. Let's drop these wells with the incorrect value and keep the one within the right Tanzania longitude/latitude range.

### Construction Year

In [26]:
#Check the construction years
clean_train['construction_year'].value_counts()

0       14489
2008     2426
2009     2306
2010     2011
2007     1423
2006     1329
2000     1252
2003     1191
2011     1032
1978      997
2004      987
2002      954
2005      879
1999      877
2012      836
1998      789
1995      782
1985      767
1984      710
1982      695
1972      679
1974      621
1990      615
1994      595
1980      581
1996      571
1992      512
1988      491
2001      476
1997      476
1993      463
1983      457
1975      402
1986      391
1976      367
1970      297
1989      293
1991      278
1987      268
1981      217
1973      183
1979      161
1977      153
2013      134
1971      128
1963       83
1967       80
1968       57
1969       55
1960       39
1962       27
1964       27
1961       20
1965       17
1966       16
Name: construction_year, dtype: int64

About a third of our data is has the construction_year equaling to 0. We will keep it as is for now.

### Installer

In [27]:
clean_train['installer'].nunique()

1757

In [28]:
clean_train['installer'].value_counts().head(50)

DWE                           13879
Government                     1283
RWE                            1007
Commu                           893
KKKT                            884
DANIDA                          861
TCRS                            668
Central government              616
CES                             610
DANID                           552
District Council                538
Community                       528
Hesawa                          523
LGA                             400
District council                386
WEDECO                          379
HESAWA                          366
Gover                           352
World vision                    345
0                               344
TASAF                           328
TWESA                           306
WU                              301
AMREF                           284
Dmdd                            284
ACRA                            277
SEMA                            249
OXFAM                       

Taking a look into our installer column, there seems to be a lot of different installers, but significant variations in spelling as well. Let's try to clean some of that up.

In [29]:
#Convert Installer Spelling Variations into Same Installer
clean_train['installer'].replace(('Gover', 'Gove', 'GOVER', 'Government Of Tanzania', 'GOVERNMENT', 
                                                         'GOVERN', 'Tanzania Government'), ('Government'))

clean_train['installer'].replace(('Dwe', 'DW', 'TWE', 'dwe', 'DWE/Anglican church', 'RWE'), ('DWE'), inplace=True)

clean_train['installer'].replace(('Unicef', 'Unisef'), ('UNICEF'), inplace=True)

clean_train['installer'].replace(('Kkkt _ Konde And Dwe', 'Kkkt', 'KKKT _ Konde and DWE', 
                                                         'KKKT MAREU', 'KKKT Canal'), ('KKKT'), inplace=True)

clean_train['installer'].replace(('Commu', 'commu', 'Communit'), ('Community'), inplace=True)

clean_train['installer'].replace(('Centre', 'Centr', 'Central govt', 'Central government', 
                                                         'Central Gover'), ('Central Government'), inplace=True)

clean_train['installer'].replace(('District council', 'Distri'), ('District Council'), inplace=True)

clean_train['installer'].replace(('World vision', 'world vision', 'WORLD VISION', 
                                                         'World Vission'), ('World Vision'), inplace=True)

clean_train['installer'].replace(('FinW', 'Fini Water ', 'FINI WATER', 'FIN WATER', 'fin water', 
                                                         'Finwater'), ('Fini Water'), inplace=True)

clean_train['installer'].replace(('RC', 'RC CHURCH', 'RC mission', 'RC Mis', 'RC church/Central Gover'), ('RC Church'), inplace=True)

clean_train['installer'].replace(('World'), ('World Bank'), inplace=True)

clean_train['installer'].replace(('Is', 'IS'), ('ISF'), inplace=True)

clean_train['installer'].replace(('Amref'), ('AMREF'), inplace=True)

clean_train['installer'].replace(('WATER AID', 'water aid'), ('Water Aid'), inplace=True)

clean_train['installer'].replace(('PRIVATE INSTITUTIONS', 'PRIV', 'PRIVATE'), ('Private'), inplace=True)

clean_train['installer'].replace(('Village Technician', 'Village Community', 'Village Council',
                                                         'Village govt', 'Village Govt', ' VILLAGE COMMUNITY', 
                                                         'Sekei village community', 'VILLAGE COUNCIL', 'Nduku village', 
                                                         'Village Counil', 'Village Government', 'Village community', 
                                                         'VILLAGE WATER COMMISSION', 'Village  Council'), ('Villagers'), inplace=True)

clean_train['installer'].replace(('Kuwaiti', 'kuwait'), ('Kuwait'), inplace=True)

clean_train['installer'].replace(('0', 'O'), ('Unknown'), inplace=True)

clean_train['installer'].replace(('Dmdd', 'dmdd'), ('DMDD'), inplace=True)

clean_train['installer'].replace(('Hesawa', 'hesawa', 'HE'), ('HESAWA'), inplace=True)

clean_train['installer'].replace(('Ministry of water engineer', 'Consulting Engineer'), 
                                                        ('Other Engineer'), inplace=True)

clean_train['installer'].replace(('DANID'), ('DANIDA'), inplace=True)

clean_train['installer'].replace(('JAICA', 'JICA', 'Jica', 'Jeica', 'JAICA CO'), ('Jaica'), inplace=True)

In [30]:
clean_train['installer'].value_counts().head(20)

DWE                   15224
Community              1464
DANIDA                 1413
Government             1283
District Council       1105
KKKT                   1053
Central Government     1017
HESAWA                  996
TCRS                    668
CES                     610
World Vision            586
Fini Water              538
LGA                     400
AMREF                   392
WEDECO                  379
DMDD                    367
RC Church               367
Villagers               366
Gover                   352
Unknown                 348
Name: installer, dtype: int64

After cleaning our installers up a bit, we see quite a change in the numbers, however DWE (District Water Engineer) seems to be the top installer for the wells within our dataset.

## Export Dataset

In [31]:
clean_train.to_csv('clean_train.csv')